In [13]:
# import python modules

import pandas as pd
import datetime 
import requests
import os
import json
from pandas.io.json import json_normalize

from dotenv import load_dotenv
load_dotenv()

import coinbasepro as cbp
from coinbase.wallet.client import Client

%matplotlib inline

In [14]:
# read API key from .env file

cb_public_key = os.getenv("CB_PUBLIC_KEY")
cb_secret_key = os.getenv("CB_SECRET_KEY")
client = Client(cb_public_key, cb_secret_key, api_version='YYYY-MM-DD')

In [15]:
# Create a function to pull one year data from CoinBase PRO API


def datapull(startDay, endDay, myCrypto):
    
    params = {"start" : startDay, "end" : endDay, "granularity" : "86400"}
    
    url = (f" https://api.pro.coinbase.com/products/{myCrypto}-USD/candles")

    requests.get(url)
    response_data = requests.get(url, params=params)
    response_content = response_data
    print(response_content)
    data = response_data.json()
        

    data_df = pd.DataFrame(data)
    data_df.columns = ["Date", "Low", "High", "Open", "Close", "Volume"]
    data_df["Date"]= pd.to_datetime(data_df['Date'],unit='s')
    
    return data_df

# Select crypto ticker to pull data for. Because CoinbasePro allows only 300 datapoints in one pull, multiple pulls with data adjustment may be needed.

crypyto ="BTC"

batch1 = datapull("2020-01-05T12:00:00","2020-10-30T12:00:00",crypyto)

batch2 = datapull("2019-10-05T12:00:00","2020-01-05T12:00:00",crypyto)

batch_combined_df = pd.concat([batch1,batch2], axis="rows", join="inner")

# Check for duplicate rows

batch_combined_df.drop_duplicates()

# Select one year data

batch_combined_df = batch_combined_df.iloc[0:365]

# Print output to csv file

#batch_combined_df.to_csv(f"output_batch_combined_{crypyto}.csv")

# Print crypto data in pandas dataframe format

batch_combined_df.head()

<Response [200]>
<Response [200]>


,Date,Low,High,Open,Close,Volume
0,2020-10-30,13131.31,13687.82,13460.00,13571.51,15664.290475
1,2020-10-29,12976.29,13650.00,13281.06,13460.00,15796.076677
2,2020-10-28,12891.57,13863.87,13740.85,13281.05,21073.376678
3,2020-10-27,13061.28,13793.72,13068.79,13698.18,17267.439678
4,2020-10-26,12785.00,13250.00,13045.02,13068.79,12275.358260
